In [1]:
from ollama import chat
from ollama import ChatResponse
import torch
import json
import os

In [2]:
def load_json(filepath):
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {filepath}")
        return None

In [1]:
def save_json(data, filepath):
    try:
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=4)
        print(f"Data saved successfully to {filepath}")
    except Exception as e:
        print(f"Error saving data to {filepath}: {e}")

In [4]:
system_prompt_dict = {}

In [5]:
system_prompt_dict["TypeScript"] = """
You are an expert TypeScript code analyst. Your task is to dissect a given TypeScript code snippet and provide a structured JSON response detailing all of its components and overall functionality. You must strictly adhere to the following JSON format:
{
  "output": {
    "programming_language": "TypeScript",
    "components": [
      {
        "component_type": "<TYPE_OF_TYPESCRIPT_COMPONENT>",
        "component_name": "<NAME_OF_COMPONENT_IF_APPLICABLE>",
        "component_code": "<THE_ACTUAL_TYPESCRIPT_CODE_OF_THE_COMPONENT>",
        "component_description": "<DETAILED_DESCRIPTION_OF_COMPONENT_FUNCTIONALITY>"
      },
      { /* ... more components ... */ }
    ],
    "overall_description": "<DETAILED_SUMMARY/DESCRIPTION_OF_THE_ENTIRE_TYPESCRIPT_CODE_FUNCTIONALITY>"
  }
}

**Instructions:**
1. Identify the Programming Language: The script is written in TypeScript.
2. Component Types: Use appropriate component types, for example, INTERFACE_DEFINITION, TYPE_ALIAS, CLASS_DEFINITION, FUNCTION_DEFINITION, METHOD_DEFINITION, GENERIC_TYPE, MODULE_IMPORT, DECORATOR, VARIABLE_DECLARATION, ENUM_DEFINITION, ASYNC_FUNCTION, PROMISE, CONDITIONAL_TYPE.
3. Component Names: Provide the correct identifier for each component (function name, class name, interface name, variable name, etc.), or NULL if not applicable.
4. Component Code: Include the complete, unmodified TypeScript code for each component.
5. Component Descriptions: Provide a detailed, technical explanation of what each component does, including TypeScript-specific features like static typing, interfaces, generics, and type inference.
6. Overall Description: Provide a detailed summary of the entire TypeScript code, explaining its purpose, architecture, and how components interact.
7. Strict JSON Output: Your ENTIRE response must be ONLY the valid JSON object. Do not include any explanations, introductions, or additional text outside the JSON structure.

Analyze the following TypeScript code properly and return ONLY the JSON response with no additional text:
"""

In [6]:
def string_to_json(input_string):
    try:
        data = json.loads(input_string)
        return data, None
    except json.JSONDecodeError as e:
        return None, str(e)

In [7]:
def make_data(data, system_prompt):
    output_data = {}

    for key, value in data.items():
        model_name = "qwen2.5-coder:32b"

        response: ChatResponse = chat(model=model_name, messages=[
            {
                'role': 'system',
                'content': system_prompt,
            },
            { 
                'role': 'user',
                'content': value,
            }
        ])

        output_data[key] = {
            "input" : value,
            "output" : response['message']['content']
        }

        print("Processed:", key)
            
    return output_data

In [8]:
data = load_json("typescript_train.json")

num_keys = 25
if not isinstance(data, dict):
    raise TypeError("Input must be a dictionary.")

if not isinstance(num_keys, int) or num_keys < 0:
    raise ValueError("num_keys must be a non-negative integer.")

sliced_dict = {}
count = 0
for key, value in data.items():
    if count < num_keys:
        sliced_dict[key] = value
        count += 1
    else:
        break

del data
del count

In [9]:
print("Length of sliced dictionary:", len(sliced_dict))

Length of sliced dictionary: 25


In [10]:
output = make_data(sliced_dict, system_prompt_dict["TypeScript"])

Processed: typescript_0
Processed: typescript_1
Processed: typescript_2
Processed: typescript_3
Processed: typescript_4
Processed: typescript_5
Processed: typescript_6
Processed: typescript_7
Processed: typescript_8
Processed: typescript_9
Processed: typescript_10
Processed: typescript_11
Processed: typescript_12
Processed: typescript_13
Processed: typescript_14
Processed: typescript_15
Processed: typescript_16
Processed: typescript_17
Processed: typescript_18
Processed: typescript_19
Processed: typescript_20
Processed: typescript_21
Processed: typescript_22
Processed: typescript_23
Processed: typescript_24


In [11]:
for k, v in output.items():
    print(f"Input: \n{v['input']}\n")
    print(f"Output: \n{v['output']}\n")
    print("-" * 80)

Input: 
import { WebpackPlugin } from './types/plugin';
declare const Plugin: WebpackPlugin;
export = Plugin;


Output: 
{
  "output": {
    "programming_language": "TypeScript",
    "components": [
      {
        "component_type": "MODULE_IMPORT",
        "component_name": "WebpackPlugin",
        "component_code": "import { WebpackPlugin } from './types/plugin';",
        "component_description": "This line imports the 'WebpackPlugin' type from a local module located at './types/plugin'. The imported type is likely an interface or a class that defines the structure and behavior of a plugin for webpack."
      },
      {
        "component_type": "VARIABLE_DECLARATION",
        "component_name": "Plugin",
        "component_code": "declare const Plugin: WebpackPlugin;",
        "component_description": "This line declares a constant variable named 'Plugin' with the type 'WebpackPlugin'. The 'declare' keyword indicates that the variable is defined elsewhere, possibly in another TypeSc

In [12]:
import pickle
# Save the output to a pickle file
with open("data.pickle", "wb") as file:
    pickle.dump(output, file)

In [2]:
import pickle
# Load the output from the pickle file
with open("data.pickle", "rb") as file:
    loaded_output = pickle.load(file)
print("Loaded output from pickle file:")
for i, (k, v) in enumerate(loaded_output.items()):
    print(f"Input {i}: \n{v['input']}\n")
    print(f"Output {i}: \n{v['output']}\n")
    print("-" * 80)

Loaded output from pickle file:
Input 0: 
import { WebpackPlugin } from './types/plugin';
declare const Plugin: WebpackPlugin;
export = Plugin;


Output 0: 
{
  "output": {
    "programming_language": "TypeScript",
    "components": [
      {
        "component_type": "MODULE_IMPORT",
        "component_name": "WebpackPlugin",
        "component_code": "import { WebpackPlugin } from './types/plugin';",
        "component_description": "This line imports the 'WebpackPlugin' type from a local module located at './types/plugin'. The imported type is likely an interface or a class that defines the structure and behavior of a plugin for webpack."
      },
      {
        "component_type": "VARIABLE_DECLARATION",
        "component_name": "Plugin",
        "component_code": "declare const Plugin: WebpackPlugin;",
        "component_description": "This line declares a constant variable named 'Plugin' with the type 'WebpackPlugin'. The 'declare' keyword indicates that the variable is defined e

In [6]:
loaded_output["typescript_16"]["output"] = loaded_output["typescript_16"]["output"][:loaded_output["typescript_16"]["output"].index("<|im_start|><|im_start|>")-2]
loaded_output["typescript_16"]["output"] = loaded_output["typescript_16"]["output"] + "\n}"

In [4]:
loaded_output["typescript_9"]["output"] = loaded_output["typescript_9"]["output"] + "\n}"

In [5]:
loaded_output["typescript_12"]["output"] = loaded_output["typescript_12"]["output"] + "\n}"

In [7]:
# After manual verification

final_data = {}
remove_ind_num = [3, 4, 5, 22]
for i, (k, v) in enumerate(loaded_output.items()):
    if i not in remove_ind_num:
        print(f"Keeping index {i} in output.")
        final_data[k] = v

Keeping index 0 in output.
Keeping index 1 in output.
Keeping index 2 in output.
Keeping index 6 in output.
Keeping index 7 in output.
Keeping index 8 in output.
Keeping index 9 in output.
Keeping index 10 in output.
Keeping index 11 in output.
Keeping index 12 in output.
Keeping index 13 in output.
Keeping index 14 in output.
Keeping index 15 in output.
Keeping index 16 in output.
Keeping index 17 in output.
Keeping index 18 in output.
Keeping index 19 in output.
Keeping index 20 in output.
Keeping index 21 in output.
Keeping index 23 in output.
Keeping index 24 in output.


In [8]:
final_data.keys()

dict_keys(['typescript_0', 'typescript_1', 'typescript_2', 'typescript_6', 'typescript_7', 'typescript_8', 'typescript_9', 'typescript_10', 'typescript_11', 'typescript_12', 'typescript_13', 'typescript_14', 'typescript_15', 'typescript_16', 'typescript_17', 'typescript_18', 'typescript_19', 'typescript_20', 'typescript_21', 'typescript_23', 'typescript_24'])

In [9]:
# Save the output to a JSON file
import json
output_json_path = "final_typescript_data.json"
try:
    save_json(final_data, output_json_path)
except Exception as e:
    print(f"Error saving final data to JSON: {e}")

Data saved successfully to final_typescript_data.json
